<a href="https://colab.research.google.com/github/gmauricio-toledo/matematicas-aprendizaje-automatico/blob/main/Notebooks/UMAP_EOMFII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UMAP: Uniform Manifold Approximation and Projection

* ArXiv: [https://arxiv.org/abs/1802.03426](https://arxiv.org/abs/1802.03426)
* GitHub: [https://github.com/lmcinnes/umap](https://github.com/lmcinnes/umap)
* Tutorial: [https://umap-learn.readthedocs.io/en/latest/#](https://umap-learn.readthedocs.io/en/latest/#)


Es un método de **reducción de dimensionalidad** que:

- Se fundamenta en conceptos de geometría diferencial y topología algebraica
- **Preserva "la" estructura local y global** de los datos
- En general, es más rápido que t-SNE
- Provee "buenas" visualizaciones

Usos:

- **Visualización**: Datos de alta dimensión $\to$ 2D/3D interpretables
- **Preprocessing**: Para clustering o clasificación
- **Exploración**: Descubrir patrones "ocultos" en datos
- **Interpretabilidad**: Entender "la" estructura de datos complejos

In [ ]:
!pip install umap-learn seaborn

In [ ]:
from sklearn.preprocessing import StandardScaler

import umap.umap_ as umap
import numpy as np
import matplotlib.pyplot as plt

## "U" Porque: Uniform Distribution of Data on a Manifold

In [ ]:
# Ejemplo 1: La Tierra es una variedad 2D que se puede encajar en R^3
fig = plt.figure(figsize=(15, 5))

# Subfigura 1: Vista global (curva)
ax1 = fig.add_subplot(121, projection='3d')
u = np.linspace(0, 2 * np.pi, 20)
v = np.linspace(0, np.pi, 20)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))
ax1.plot_surface(x, y, z, alpha=0.7, color='orange')
ax1.set_title('Vista Global\n(La Tierra es CURVA)')

# Subfigura 2: Vista local (plana)
ax2 = fig.add_subplot(122)
ax2.add_patch(plt.Rectangle((0, 0), 1, 1, facecolor='orange', alpha=0.7))
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.set_title('Vista Local\n(Tu entorno parece PLANO)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
def crear_datos():
    # Datos globalmente uniformes
    global_uniform = np.random.uniform(-3, 3, (400, 2))

    # Datos localmente uniformes (3 clusters con densidades diferentes)
    cluster1 = np.random.multivariate_normal([-2, -2], [[0.2, 0], [0, 0.2]], 120)   # Muy denso
    cluster2 = np.random.multivariate_normal([0, 0], [[0.8, 0], [0, 0.8]], 150)     # Menos denso
    cluster3 = np.random.multivariate_normal([3, 2], [[0.4, 0], [0, 0.4]], 100)     # Densidad media

    local_data = np.vstack([cluster1, cluster2, cluster3])
    labels = np.hstack([np.zeros(120), np.ones(150), np.full(100, 2)])

    return global_uniform, local_data, labels

global_data, local_data, labels = crear_datos()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].scatter(global_data[:, 0], global_data[:, 1], alpha=0.6, s=20)
axes[0].set_title('Globalmente Uniforme en R^2', fontweight='bold')

axes[1].scatter(local_data[:, 0], local_data[:, 1], c=labels, cmap='viridis', s=20)
axes[1].set_title('Localmente Uniforme en R^2 (UMAP)', fontweight='bold')

plt.tight_layout()
plt.show()

# Ejemplos

In [ ]:
from sklearn.datasets import (make_blobs, make_circles, make_moons,
                             load_digits, fetch_olivetti_faces)

In [ ]:
#Crea diversos datasets para UMAP

def crear_datasets_ejemplo():
    datasets = {}

    # 1. Círculos concéntricos
    x_circles, y_circles = make_circles(n_samples=400, factor=0.3, noise=0.1, random_state=42)
    datasets['circles'] = (x_circles, y_circles, 'Círculos Concéntricos')

    # 2. Lunas entrelazadas
    x_moons, y_moons = make_moons(n_samples=400, noise=0.1, random_state=42)
    datasets['moons'] = (x_moons, y_moons, 'Lunas Entrelazadas')

    # 3. Espiral 3D
    n_points = 400
    t = np.linspace(0, 4*np.pi, n_points)
    x_spiral = t * np.cos(t)
    y_spiral = t * np.sin(t)
    z_spiral = t
    X_spiral = np.column_stack([x_spiral, y_spiral, z_spiral])
    y_spiral = (t > 2*np.pi).astype(int)  # Dos grupos por altura
    datasets['spiral'] = (X_spiral, y_spiral, 'Espiral 3D')

    # 4. Dataset de alta dimensión
    x_high, y_high = make_blobs(n_samples=500, centers=5, n_features=50,
                               random_state=42, cluster_std=2.0)
    datasets['high_dim'] = (x_high, y_high, 'Alta Dimensión (50D)')

    # 5. Datos con ruido
    x_noise = np.random.randn(300, 20)
    # Añadir estructura oculta en primeras 3 dimensiones
    x_noise[:100, :3] += [2, 2, 2]  # Grupo 1
    x_noise[100:200, :3] += [-2, 2, -2]  # Grupo 2
    x_noise[200:, :3] += [0, -3, 0]  # Grupo 3
    y_noise = np.array([0]*100 + [1]*100 + [2]*100)
    datasets['noisy'] = (x_noise, y_noise, 'Datos con Ruido')

    return datasets

# Crear todos los datasets
datasets = crear_datasets_ejemplo()

In [ ]:
# Visualizar datasets originales
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, (key, (x, y, title)) in enumerate(datasets.items()):
    if i >= 6:
        break

    if x.shape[1] == 2:
        axes[i].scatter(x[:, 0], x[:, 1], c=y, cmap='tab10', alpha=0.7)
        axes[i].set_xlabel('Dimensión 1')
        axes[i].set_ylabel('Dimensión 2')
    elif x.shape[1] == 3:
        # Para 3D, mostrar proyección
        axes[i].scatter(x[:, 0], x[:, 1], c=y, cmap='tab10', alpha=0.7)
        axes[i].set_xlabel('Dimensión 1')
        axes[i].set_ylabel('Dimensión 2')
    else:
        # Para alta dimensión, mostrar PCA
        from sklearn.decomposition import PCA
        pca = PCA(n_components=2)
        x_pca = pca.fit_transform(x)
        axes[i].scatter(x_pca[:, 0], x_pca[:, 1], c=y, cmap='tab10', alpha=0.7)
        axes[i].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
        axes[i].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')

    axes[i].set_title(f'{title}\n({x.shape[0]} puntos, {x.shape[1]} dim)')
    axes[i].grid(True, alpha=0.3)

# Ocultar último subplot si no se usa
if len(datasets) < 6:
    axes[5].axis('off')

plt.tight_layout()
plt.show()

print(f"{len(datasets)} datasets de ejemplo")
for key, (x, y, title) in datasets.items():
    print(f"   {key}: {title} - {x.shape[0]} muestras, {x.shape[1]} características")

In [ ]:
#UMAP-Básico: Aplicar UMAP a todos los datasets con parámetros por defecto
umap_results = {}

for key, (X, y, title) in datasets.items():
    print(f"Procesando {title}...")

    # Normalizar los datos si es necesario
    if X.shape[1] > 3:  # Solo para alta dimensión
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
    else:
        X_scaled = X

    # Aplicar UMAP
    reducer = umap.UMAP(random_state=42)
    embedding = reducer.fit_transform(X_scaled)

    umap_results[key] = {
        'original': X_scaled,
        'embedding': embedding,
        'labels': y,
        'title': title,
        'reducer': reducer
    }

print("UMAP aplicado a todos los datasets")

In [ ]:
# Crear una visualización comparativa
fig, axes = plt.subplots(2, len(datasets), figsize=(4*len(datasets), 8))

for i, (key, result) in enumerate(umap_results.items()):
    X_orig = result['original']
    embedding = result['embedding']
    y = result['labels']
    title = result['title']

    # Fila superior: datos originales (o PCA si >2D)
    if X_orig.shape[1] == 2:
        axes[0, i].scatter(X_orig[:, 0], X_orig[:, 1], c=y, cmap='tab10', alpha=0.7)
        axes[0, i].set_title(f'Original: {title}')
    else:
        # PCA para visualización
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X_orig)
        axes[0, i].scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='tab10', alpha=0.7)
        axes[0, i].set_title(f'PCA: {title}')

    axes[0, i].grid(True, alpha=0.3)

    # Fila inferior: UMAP
    scatter = axes[1, i].scatter(embedding[:, 0], embedding[:, 1], c=y, cmap='tab10', alpha=0.7)
    axes[1, i].set_title(f'UMAP: {title}')
    axes[1, i].set_xlabel('UMAP 1')
    axes[1, i].set_ylabel('UMAP 2')
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Swiss Roll

El Swiss Roll es un dataset clásico para demostrar técnicas de reducción
de dimensionalidad no lineal. Es una superficie 2D enrollada en 3D.

¿Por qué es importante?
- Demuestra cómo UMAP "desenrolla" estructuras no lineales
- Muestra la diferencia entre PCA (lineal) y UMAP (no lineal)
- Es un manifold 2D embebido en espacio 3D

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA

In [ ]:
# Generar Swiss Roll
n_samples = 2000
noise = 0.1

X_swiss, t = make_swiss_roll(n_samples=n_samples, noise=noise, random_state=42)

print(f"Swiss Roll generado:")
print(f"- Muestras: {n_samples}")
print(f"- Dimensiones: {X_swiss.shape[1]} (3D)")
print(f"- Ruido: {noise}")
print(f"- Variable t: representa la posición en el rollo (1D continua)")
print(f"- Rango de t: [{t.min():.2f}, {t.max():.2f}]")

# Visualizar Swiss Roll original en 3D
fig = plt.figure(figsize=(15, 5))

# Vista 1: Perspectiva estándar
ax1 = fig.add_subplot(131, projection='3d')
scatter1 = ax1.scatter(X_swiss[:, 0], X_swiss[:, 1], X_swiss[:, 2],
                      c=t, cmap='viridis', s=20, alpha=0.8)
ax1.set_xlabel('X', fontweight='bold')
ax1.set_ylabel('Y', fontweight='bold')
ax1.set_zlabel('Z', fontweight='bold')
ax1.set_title('Swiss Roll Original\n(Vista 1)', fontsize=12, fontweight='bold')
ax1.view_init(elev=10, azim=70)
plt.colorbar(scatter1, ax=ax1, label='Posición en el rollo (t)', shrink=0.6)

# Vista 2: Vista superior
ax2 = fig.add_subplot(132, projection='3d')
scatter2 = ax2.scatter(X_swiss[:, 0], X_swiss[:, 1], X_swiss[:, 2],
                      c=t, cmap='viridis', s=20, alpha=0.8)
ax2.set_xlabel('X', fontweight='bold')
ax2.set_ylabel('Y', fontweight='bold')
ax2.set_zlabel('Z', fontweight='bold')
ax2.set_title('Swiss Roll Original\n(Vista Superior)', fontsize=12, fontweight='bold')
ax2.view_init(elev=90, azim=0)

# Vista 3: Vista lateral
ax3 = fig.add_subplot(133, projection='3d')
scatter3 = ax3.scatter(X_swiss[:, 0], X_swiss[:, 1], X_swiss[:, 2],
                      c=t, cmap='viridis', s=20, alpha=0.8)
ax3.set_xlabel('X', fontweight='bold')
ax3.set_ylabel('Y', fontweight='bold')
ax3.set_zlabel('Z', fontweight='bold')
ax3.set_title('Swiss Roll Original\n(Vista Lateral)', fontsize=12, fontweight='bold')
ax3.view_init(elev=0, azim=0)

plt.suptitle('Swiss Roll en 3D - Diferentes Perspectivas',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# PCA: MÉTODO LINEAL (Para comparación)
# ============================================================================

print("\n" + "="*70)
print("Aplicar PCA (Método Lineal)")
print("="*70 + "\n")

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_swiss)

print("PCA aplicado:")
print(f"- Varianza explicada: {pca.explained_variance_ratio_.sum()*100:.2f}%")
print(f"- PC1: {pca.explained_variance_ratio_[0]*100:.2f}%")
print(f"- PC2: {pca.explained_variance_ratio_[1]*100:.2f}%")

In [ ]:
# Configuración 1: UMAP estándar
print("Configuración 1: UMAP estándar...")
reducer_standard = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    n_components=2,
    random_state=42,
    verbose=False
)

X_umap_standard = reducer_standard.fit_transform(X_swiss)

# Configuración 2: UMAP con más vecinos (preserva estructura global)
print("\nConfiguración 2: UMAP con más vecinos (n_neighbors=50)...")
reducer_global = umap.UMAP(
    n_neighbors=50,
    min_dist=0.1,
    n_components=2,
    random_state=42,
    verbose=False
)

X_umap_global = reducer_global.fit_transform(X_swiss)

# Configuración 3: UMAP con menos vecinos (preserva estructura local)
print("\nConfiguración 3: UMAP con menos vecinos (n_neighbors=5)...")
reducer_local = umap.UMAP(
    n_neighbors=5,
    min_dist=0.1,
    n_components=2,
    random_state=42,
    verbose=False
)

X_umap_local = reducer_local.fit_transform(X_swiss)


In [ ]:
# ============================================================================
# COMPARACIÓN VISUAL: PCA vs UMAP
# ============================================================================

fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Subplot 1: PCA
scatter1 = axes[0, 0].scatter(X_pca[:, 0], X_pca[:, 1],
                             c=t, cmap='viridis', s=20, alpha=0.8)
axes[0, 0].set_title('PCA (Lineal)\n NO desenrolla el Swiss Roll',
                    fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('PC 1', fontweight='bold')
axes[0, 0].set_ylabel('PC 2', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0, 0])

# Subplot 2: UMAP estándar
scatter2 = axes[0, 1].scatter(X_umap_standard[:, 0], X_umap_standard[:, 1],
                             c=t, cmap='viridis', s=20, alpha=0.8)
axes[0, 1].set_title('UMAP Estándar (n_neighbors=15)\n Desenrolla el rollo',
                    fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('UMAP 1', fontweight='bold')
axes[0, 1].set_ylabel('UMAP 2', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[0, 1])

# Subplot 3: UMAP global
scatter3 = axes[1, 0].scatter(X_umap_global[:, 0], X_umap_global[:, 1],
                             c=t, cmap='viridis', s=20, alpha=0.8)
axes[1, 0].set_title('UMAP Global (n_neighbors=50)\n Enfoque en estructura global',
                    fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('UMAP 1', fontweight='bold')
axes[1, 0].set_ylabel('UMAP 2', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
plt.colorbar(scatter3, ax=axes[1, 0])

# Subplot 4: UMAP local
scatter4 = axes[1, 1].scatter(X_umap_local[:, 0], X_umap_local[:, 1],
                             c=t, cmap='viridis', s=20, alpha=0.8)
axes[1, 1].set_title('UMAP Local (n_neighbors=5)\n Enfoque en detalles locales',
                    fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('UMAP 1', fontweight='bold')
axes[1, 1].set_ylabel('UMAP 2', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
plt.colorbar(scatter4, ax=axes[1, 1], label='Posición (t)')

plt.suptitle('Comparación: PCA vs UMAP en Swiss Roll',
             fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

In [ ]:
print("CONCLUSIONES:")
print()
print("PCA (Lineal):")
print("- NO puede desenrollar el Swiss Roll")
print("- Solo encuentra proyecciones lineales")
print()
print("UMAP (No Lineal):")
print("- Desenrolla exitosamente el Swiss Roll")
print("- Preserva la estructura topológica")
print("- Revela la dimensionalidad intrínseca (2D)")
print()
print("Aplicaciones:")
print("- Datos genómicos con estructura compleja")
print("- Señales de audio con patrones temporales")
print("- Imágenes con transformaciones continuas")
print("- Embeddings de redes neuronales")

# MNIST

Se muestra una manera de aplicar UMAP al dataset MNIST:

1. **Preparación**: Carga de datos.
2. **Aplicación**: Configuración y ejecución de UMAP.
3. **Visualización**: Análisis de los resultados en 2D.
4. **Evaluación**: Métricas de calidad del embedding.

NOTAS:
- UMAP preserva tanto estructura local como globlal.
- Los dígitos forman clusters naturales "bien" separados.
- Los parámetros `n_neighbors` y `min_dist` afectan la estructura final.

In [ ]:
from sklearn.datasets import fetch_openml

import seaborn as sns
import time

`fetch_openml` permite descargar datasets directamente desde OpenML (Open Machine Learning), el cual es un repositorio público de datasets para machine learning.

¿Por qué usar `fetch_openml`?
- Acceso directo a datasets estándar sin necesidad de descargas manuales.
- Garantiza consistencia en formato y versiones.
- Incluye metadatos del dataset automáticamente.

## Cargar MNIST:

In [ ]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, Y = mnist.data, mnist.target.astype(int)

In [ ]:
print(f"Datos: {X.shape}")
print(f"Etiquetas: {Y.shape}")
print(f"Rango de valores: {X.min()} - {X.max()}")
print(f"Clases: {np.unique(Y)}")

* **Estructura de Primeros Datos:**

In [ ]:
print("Primeras 5 etiquetas:", Y[:5])
print("Forma de primera imagen:", X[0].reshape(28, 28).shape)
print("Estadísticas de primera imagen:")
print(f"Min: {X[0].min()}")
print(f"Max: {X[0].max()}")
print(f"Media: {X[0].mean():.2f}")
print(f"Std: {X[0].std():.2f}")

### Detalles

---

**Parámetros:**

`'mnist_784'`
- Identificador único del dataset MNIST en OpenML
- El "784" hace referencia a las dimensiones: 28×28 = 784 píxeles por imagen

`version=1`
- Especifica la versión del dataset a descargar
- Versión 1 es la versión estándar

`as_frame=False`
- Formato de salida: NumPy arrays en lugar de pandas DataFrames
- `False`: Retorna carrays de NumPy (recomendable para ML)
- `True`: Retornaría pandas DataFrame (recomendable para análisis exploratorio)

---

**Separación de Datos y Etiquetas:**

Estructura del objeto `mnist`:
- `mnist.data`: Matriz de características (imágenes)
- `mnist.target`: Vector de etiquetas (dígitos)
- `mnist.DESCR`: Descripción del dataset
- `mnist.feature_names`: Nombres de las características

`X = mnist.data`
- X: Matriz de características (convención en ML)
- Forma: (70000, 784)
  - 70,000 imágenes
  - 784 características por imagen (28×28 píxeles)
- Tipo de datos: float64 por defecto
- Valores: Intensidad de píxeles (0-255)

#### `y = mnist.target.astype(int)`
- Y: Vector de etiquetas objetivo
- Forma: (70000,)
- Conversión .astype(int):
  - Las etiquetas vienen como strings ('0', '1', '2', ...)
  - Se convierten a enteros (0, 1, 2, ...) para procesamiento
  - Más eficiente para algoritmos de ML

---

**Inspección de Dimensiones**

Interpretación de `X.shape`:
  - (70000, 784) significa: 70,000 muestras (imágenes) y 784 características por muestra
  - Cada imagen 28x28 se ha "aplanado" en un vector unidimensional

Interpretación de `Y.shape`:
- (70000, ) significa: 70,000 etiquetas, una etiqueta por cada imagen. Lo que resulta en un vector unidimensional

¿Por qué 70,000 muestras?
- MNIST completo: 60,000 entrenamiento + 10,000 prueba
- `fetch_openml` descarga el dataset completo por defecto
- La separación train/test debe hacerse manualmente si se requiere

---

**Análisis del Rango de Valores (`{X.min()}` - `{X.max()}`)**

¿Qué información proporciona?
- Valores esperados: 0 - 255
- 0: píxel completamente negro (fondo)
- 255: píxel completamente blanco (tinta)
- Valores intermedios: Diferentes tonos de gris

En general, UMAP funciona mejor con datos normalizados. El rango 0-255 puede causar que características con valores altos dominen. De ser necesario, se recomienda preprocesamiento

---

**Verificación de Clases (`np.unique(Y)`)**

- Salida esperada: `[0 1 2 3 4 5 6 7 8 9]`
- 10 clases: dígitos del 0 al 9
- Balanceado: aproximadamente igual número de muestras por clase

---

**Resumen:**

- **X**: (70000, 784) - Imágenes "aplanadas"
- **Y**: (70000,) - Etiquetas de dígitos (0-9)
- **Formato**: Arrays de NumPy
- **Rango**: 0-255 (se sugiere normalización)

___

## Visualizar Imágenes de MNIST:

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
fig.suptitle('Muestras de MNIST', fontsize=12)

for i, ax in enumerate(axes.flat):
    ax.imshow(X[i].reshape(28,28), cmap='gray')
    ax.set_title(f'Dígito: {Y[i]}')
    ax.axis('off')

plt.tight_layout()
plt.show()

### Detalles

___

**Configuración de la Estructura de Visualización:**

`plt.subplots(2, 5, figsize=(12, 6))`
- `2, 5`: Crea una cuadrícula de *2 filas x 5 columnas* = 10 subplots
- `figsize=(12, 6)`: Tamaño de la figura completa en pulgadas
  - 12 pulgadas de ancho (suficiente espacio para 5 imágenes por fila)
  - 6 pulgadas de alto (altura para 2 filas)

Objetos devueltos:
- `fig`: Objeto Figure (contenedor principal)
- `axes`: Array 2D de objetos Axes (subplots individuales)
  - Forma: `axes.shape = (2, 5)`
  - Cada `axes[i, j]` es un subplot individual

---

**Configuración del Título Principal:**

`fig.suptitle()`
- Función: Añade título principal a toda la figura
- Posición: Centrado en la parte superior
- Diferencia con `plt.title()`:
  - `suptitle()`: Título de toda la figura
  - `title()`: Título de un subplot individual
- `fontsize=16`: Tamaño de fuente grande para visibilidad

---

**Iteración a través de los Subplots. Desglose de la Iteración:**

`axes.flat`
- Iterador que "aplana" el array 2D de axes
- Conversión: `axes[2,5]` $\to$ secuencia lineal de 10 elementos
- Orden: Fila por fila (row-major order)

  `axes[0,0]` $\to$ `axes[0,1]` $\to$ ... $\to$ `axes[0,4]` $\to$
  `axes[1,0]` $\to$ `axes[1,1]` $\to$ ... $\to$ `axes[1,4]`

`enumerate()`
- Añade un contador automático a la iteración
- Retorna: Tuplas `(índice, valor)`
- Ventaja: Acceso simultáneo al índice y al objeto axes

---

**Transformación y Visualización de Imágenes. Análisis Detallado de Cada Componente:**

`X[i]`
- i-ésima imagen del dataset
- `(784,)` - Vector unidimensional
- Valores de píxeles 0-255

`.reshape(28, 28)`
- Redimensiona el vector 1D a matriz 2D
- Transformación: `(784,)` → `(28, 28)`
- Orden: Row-major (fila por fila)
- Validación matemática: 784 = 28 x 28

`cmap='gray'`
- Escala de grises
- Mapeo:
  - Valor 0 $\to$ Negro (fondo)
  - Valor 255 $\to$ Blanco (tinta)
  - Valores intermedios $\to$ Tonos de gris
- ¿Por qué 'gray'?: MNIST son imágenes en escala de grises originalmente

---

**Configuración de Títulos Individuales. Componentes:**

`Y[i]`
- Etiqueta correspondiente a la i-ésima imagen
- Valor: Entero 0-9 (clase del dígito)
- Correspondencia: `X[i]` (imagen) $\iff$ `Y[i]` (etiqueta)

`f'Dígito: {y[i]}'`
- f-string: Formato moderno de strings en Python
- Resultado: "Dígito: 0", "Dígito: 1",...

`ax.set_title()`
- Establece un título para cada subplot individual
- Posición: Arriba del subplot
- Diferencia con `suptitle()`: Específico para cada imagen

---

**Paso 6: Limpieza de Ejes.**

¿Por qué eliminar los ejes? No son informativos, interfieren con la imagen, los labels ocupan espacio innecesario.

`ax.axis('off')`
- Elimina: Ticks, labels, y líneas de los ejes
- Conserva: La imagen y el título
- Resultado: Visualización más limpia y enfocada

---

**Optimización del Layout:**

¿Qué hace `tight_layout()`? Espacios óptimos entre subplots. Títulos y labels no se superponen. Máximo uso del área disponible

---

**Renderizado Final:**

`plt.show()`

1. Compilación: Matplotlib procesa todos los comandos
2. Renderizado: Genera la imagen final
3. Visualización: Muestra en pantalla (Jupyter/Colab)
___

**Resumen:**

- Datos visualizados y comprendidos
- Estructura de imágenes confirmada
- Listo para preprocesamiento y análisis con UMAP

___

## Preprocesamiento de Datos:

In [ ]:
n_samples = 70000  # Puedes usar menos muestras para un procesamiento más rápido
np.random.seed(42)
indices = np.random.choice(len(X), n_samples, replace=False)
X_subset = X[indices]
Y_subset = Y[indices]

print(f"Usando {n_samples} muestras para el análisis")

#Normalización de datos (recomendado para UMAP)
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X_subset)  #OJO: cambiar variable `X_subset` por `X_scaled`en todas las celdas!
#print("Datos normalizados correctamente")

### Detalles

___


**Razones para la Selección de Subconjunto de Datos (`n_samples`):**

- Consideraciones Computacionales
  - Dataset completo: 70,000 imágenes
  - Memoria RAM: ~210 MB para datos originales + embeddings
  - Tiempo de procesamiento: UMAP puede tomar 5-30 minutos en dataset completo
  - Recursos limitados: Google Colab tiene limitaciones de tiempo y memoria

- Desarrollo y Prototipado
  - Iteración rápida: Probar diferentes parámetros más eficientemente
  - Debugging: Detectar de manera "más fácil" problemas con menos datos
  - Experimentación: Validar el "pipeline" antes del procesamiento completo

- Consideraciones Pedagógicas
  - Demostración: Resultados visibles en tiempo razonable
  - Comprensión: Enfocarse en conceptos sin esperar largos procesamientos

- ¿Cuándo usar el dataset completo?
  - Análisis final y publicación
  - Cuando se requiere máxima precisión
  - Con recursos computacionales suficientes

---

**Generación de Índices Aleatorios:**

`np.random.choice()`
- Selección aleatoria de elementos
- Tipo: Muestreo estadístico controlado

`len(X)` (población)
- Valor: 70,000 (tamaño total del dataset)
- Significado: Rango de índices posibles [0, 69999]
- Tipo: Entero que define el espacio de muestreo

`n_samples` (tamaño de muestra)
- Valor: 70,000 (número de muestras deseadas)
- Proporción: ~$x$% del dataset original
- Balance: Suficiente para representatividad, manejable computacionalmente

`replace=False` (sin reemplazo)
- Cada muestra puede ser seleccionada solo una vez
- Resultado: Sin duplicados en la selección
- Alternativa: `replace=True` permitiría duplicados
- Importancia: Mantiene independencia estadística

En su caso, ¿por qué un muestreo aleatorio?
1. Representatividad: Preserva la distribución original de clases
2. Imparcialidad: Evita sesgos de selección
3. Generalización: Resultados aplicables al dataset completo

---

**Extracción de Subconjunto:**

`X[indices]`
- Tipo de operación: Indexación avanzada de NumPy (fancy indexing)
- Entrada: Array de índices `[1023, 45678, 234, ...]`
- Salida: Subarray con filas seleccionadas
- Transformación (ejemplo con n_samples=10000): `(70000, 784)` $\to$ `(10000, 784)`

`Y[indices]`
- Operación paralela: Mismos índices para mantener correspondencia
- Transformación (ejemplo con n_samples=10000): `(70000,)` → `(10000,)`
- Correspondencia: `X_subset[i]` $\iff$ `y_subset[i]`, para cada $i$.

---

**Escalador:**

¿Qué es StandardScaler? Matemáticamente, es la transformación dada por
$$
\text{X_scaled} = \frac{(X - \mu)}{\sigma},
$$
donde $\mu$ es la media de cada característica y $\sigma$ es la desviación estándar de la misma. Como resultado de la transformación se tiene:
- Media: Cero para todas las características
- Desviación estándar: 1 para todas las características
- Distribución: Mantiene la forma original, solo reescala

---

**¿Por qué Normalización para UMAP?**

Primero, algunas cuestiones que se pueden presentar sin normalización:

- Dominancia de características con valores grandes:
```python
# Ejemplo conceptual
pixel_centro = 200    # Píxel con tinta (valor alto)
pixel_esquina = 5     # Píxel de fondo (valor bajo)

# Distancia euclidiana sin normalización
dist_raw = sqrt((200-180)² + (5-3)²) = sqrt(400 + 4) = 20.1

# El píxel del centro domina el cálculo de distancia
```

- Sesgo en el cálculo de distancias:
  - UMAP usa distancias para construir el grafo de vecindarios
  - Píxeles centrales: Tienden a tener valores más altos (presencia de tinta)
  - Píxeles de borde: Generalmente son fondo (valores bajos)
  - Resultado: El algoritmo puede ignorar variaciones sutiles importantes

- Convergencia lenta:
  - UMAP usa optimización gradient descent
  - Escalas diferentes: Diferentes características requieren diferentes tasas de aprendizaje
  - Convergencia: Más lenta y menos estable sin normalización

**"Beneficios" de la normalización:**

- Equidad entre características:
  - Todas las características contribuyen igualmente
  - Variaciones sutiles se preservan
  - Mejor captura de patrones complejos

- Mejor performance del algoritmo:
  - Convergencia más rápida
  - Resultados más estables
  - Menor sensibilidad a hiperparámetros

- Interpretabilidad mejorada:
  - Distancias más significativas
  - Estructuras de cluster más claras

---

**Desglose del proceso `fit_transform`:**

Fase 1: Fit/Ajuste (`scaler.fit(X_subset)`)

- Cálculo de medias por característica:
   ```python
   μ = np.mean(X_subset, axis=0)  # Forma: (784,)
   ```

- Cálculo de desviaciones estándar:
   ```python
   σ = np.std(X_subset, axis=0)   # Forma: (784,)
   ```

Almacenamiento de parámetros:
   ```python
   scaler.mean_ = μ      # Medias aprendidas
   scaler.scale_ = σ     # Desviaciones aprendidas
   ```

**Transform/Transformación (`scaler.transform()`)**

- Aplicación de la transformación:
```python
X_scaled = (X_subset - scaler.mean_) / scaler.scale_
```

- ¿Por qué `fit_transform()` en una sola llamada?
  - Eficiencia: Una sola pasada por los datos
  - Consistencia: Garantiza que fit y transform usen los mismos datos
  - Conveniencia: Sintaxis más simple

---

**Recomendaciones para MNIST + UMAP**

`StandardScaler` es generalmente la mejor opción porque:
- Manejo del fondo: Convierte los muchos ceros a valores negativos
- Énfasis en variaciones: Amplifica diferencias sutiles en la tinta
- Compatibilidad con UMAP: UMAP funciona bien con datos centrados
- Resultados empíricos: Consistentemente produce buenos embeddings

---

## Aplicación de UMAP:

In [ ]:
reducer = umap.UMAP(
    n_neighbors=15,      # Número de vecinos cercanos
    min_dist=0.1,        # Distancia mínima entre puntos en embedding
    n_components=2,      # Dimensiones de salida (2D para visualización)
    metric='euclidean',  # Métrica de distancia
    random_state=42      # Semilla para reproducibilidad
)

# Aplicar UMAP y medir tiempo
start_time = time.time()
embedding = reducer.fit_transform(X_subset)
end_time = time.time()

In [ ]:
print(f"UMAP completado en {end_time - start_time:.2f} segundos")
print(f"'Forma' del embedding: {embedding.shape}")

### Detalles

#### `umap.umap_` (Importación de UMAP)

- Módulo principal: Contiene la implementación core de UMAP
- Convención de nombrado: `umap_` con underscore evita conflictos con el nombre del paquete
- Alias: Se importa como `umap` para simplicidad

**Alternativas de importación:**
```python
# Importación directa de la clase
from umap import UMAP

# Importación completa del módulo
import umap
```

---

**Configuración del Objeto UMAP:**

```python
reducer = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    n_components=2,
    metric='euclidean',
    random_state=42
)
```

`reducer`
- **Tipo**: Instancia de la clase UMAP
- **Estado**: No entrenado (no ha visto datos aún)

#### `n_neighbors` (Vecindarios Locales)

*Conceptualmente:*
- Número de vecinos cercanos considerados para cada punto
- Escala local-global: Balance entre estructura local y global
- Construcción del grafo: Número de conexiones por nodo

*Funcionamiento interno:*

- Construcción del grafo de vecindarios:
```python
# Pseudocódigo del proceso
for punto_i in dataset:
    # Encontrar los k vecinos más cercanos
    vecinos = encontrar_k_vecinos_mas_cercanos(punto_i, k=15)
    
    # Crear aristas con pesos basados en distancias
    for vecino_j in vecinos:
        distancia = calcular_distancia(punto_i, vecino_j)
        peso = funcion_probabilidad(distancia)
        grafo.agregar_arista(punto_i, vecino_j, peso)
```

*Efectos de diferentes valores:*

n_neighbors pequeño (5-10):
- Preserva estructura local: Detalles finos visibles
- Clusters más granulares: Subclusters evidentes
- $\times$ Estructura global fragmentada: Conexiones a larga distancia perdidas
- $\times$ Más ruido: Sensible a variaciones locales

n_neighbors medio (15-30):
- Balance óptimo: Estructura local y global preservadas
- Robustez: Menos sensible a ruido
- Generalización: Captura patrones consistentes
- Recomendación: Valor por defecto para la mayoría de casos

n_neighbors grande (50-200):
- Estructura global clara: Relaciones macro preservadas
- Smooth embeddings: Transiciones suaves entre clusters
- $\times$ Pérdida de detalles locales: Estructuras finas comprimidas
- $\times$ Over-smoothing: Posible fusión de clusters distintos

Analogía:
Imagina que estás en una ciudad desconocida:
- n_neighbors=5: Solo preguntas a los 5 vecinos más cercanos $\to$ conoces tu barrio detalladamente
- n_neighbors=15: Preguntas a 15 personas $\to$ entiendes el barrio y algo de la ciudad
- n_neighbors=50: Preguntas a 50 personas $\to$ tienes mapa general pero pierdes detalles locales

Impacto en MNIST:
```python
# Ejemplo visual del efecto
n_neighbors_values = [5, 15, 50, 100]

# Con n_neighbors=5:  Dígitos similares forman sub-clusters muy separados
# Con n_neighbors=15: Balance óptimo, clusters claros pero conectados
# Con n_neighbors=50: Visión macro, transiciones suaves entre dígitos
```

#### `min_dist` (Distancia Mínima)

*Conceptualmente:*
- Distancia mínima entre puntos en el embedding de baja dimensión
- Compactación vs. dispersión: Qué tan "apretados" están los clusters
- Solo afecta el embedding: No afecta la topología "aprendida"

*Funcionamiento interno:*

```python
# Pseudocódigo simplificado
def optimizar_embedding(grafo, min_dist):
    # Inicializar posiciones aleatorias en 2D
    posiciones = inicializar_aleatoriamente()
    
    for iteracion in range(n_epochs):
        for arista in grafo:
            punto_a, punto_b = arista
            
            # Calcular distancia actual en embedding
            dist_actual = distancia(posiciones[punto_a], posiciones[punto_b])
            
            # Si son vecinos en el grafo:
            if son_vecinos(punto_a, punto_b):
                # Fuerza atractiva (quiere acercarlos)
                if dist_actual > min_dist:
                    aplicar_fuerza_atractiva()
            else:
                # Fuerza repulsiva (quiere alejarlos)
                if dist_actual < dist_minima_no_vecinos:
                    aplicar_fuerza_repulsiva()
```

*Efectos de diferentes valores:*

min_dist pequeño (0.0 - 0.1):
- Clusters compactos**: Puntos muy juntos dentro de clusters
- Separación clara: Clusters bien diferenciados
- Visualización nítida: Estructuras más definidas
- $\times$ Posible solapamiento interno: Detalles internos comprimidos

min_dist medio (0.1 - 0.5):
- Balance visual: Ni muy compacto ni muy disperso
- Estructura interna visible: Detalles dentro de clusters preservados
- Recomendación: Valor estándar por defecto

min_dist grande (0.5 - 1.0):**
- Distribución uniforme: Puntos más espaciados
- Topología global clara: Relaciones macro visibles
- $\times$ Clusters difusos: Límites menos definidos
- $\times$ Uso ineficiente del espacio: Visualización más grande de lo necesario

Analogía:
Imagina organizar personas en una sala:
- min_dist=0.0: Personas del mismo grupo pegadas hombro con hombro
- min_dist=0.1: Espacio personal cómodo dentro de grupos
- min_dist=0.5: Cada persona tiene bastante espacio alrededor

Resumen:
```
┌───────────────────────────────────────────────────┐
│  n_neighbors: "¿Con quién debo agruparme?"        │
│  min_dist:    "¿Qué tan cerca debo estar?"        │
└───────────────────────────────────────────────────┘
```

Impacto en MNIST:
```python
min_dist=0.0:  #Clusters muy densos, como "islas" compactas
min_dist=0.1:  #Clusters definidos con estructura interna visible
min_dist=0.5:  #Clusters más difusos, transiciones suaves
```

#### `n_components` (Dimensionalidad de Salida)

*Definición:*
- Número de dimensiones en el espacio de embedding
- Reducción: 784 dimensiones $\to$ 2 dimensiones

*Opciones comunes:*

n_components=2:
- Visualización directa: Gráficas 2D fáciles de interpretar
- Exploración rápida: Scatter plots inmediatos
- $\times$ Pérdida de información: Mayor compresión
- Usos comunes: Análisis exploratorio, presentaciones, papers

n_components=3:
- Más información preservada: Menos compresión
- Visualización 3D: Gráficas rotables
- $\times$ Más complejo visualizar: Requiere herramientas 3D
- Tip: Cuando 2D no es suficientemente informativo

n_components=10-50:
- Máxima preservación: Estructura compleja preservada
- Preprocessing para ML: Input para clasificadores
- $\times$ No visualizable directamente: Requiere análisis adicional
- Usos comunes: Feature engineering, pipeline de ML

*Comparación cuantitativa:*
```python
# Información preservada (aproximado para MNIST)
n_components=2:   #~65% de varianza explicada
n_components=3:   #~75% de varianza explicada
n_components=10:  #~90% de varianza explicada
n_components=50:  #~98% de varianza explicada
```

*Consideraciones prácticas:*
```python
# Para visualización
n_components = 2  # Siempre la primera opción

# Para preprocessing de ML
n_components = min(50, n_samples // 10)  # "Regla de oro"

# Para análisis detallado
n_components = 3  # Permite visualización 3D interactiva
```

#### `metric` (Función de Distancia)

*Definición:*
- Función de distancia usada para calcular similitud entre puntos
- Observación fundamental: Determina qué significa "cercano" o "lejano"
- Fase de construcción del grafo: Usado antes de la optimización
- Por defecto: Distancia Euclidiana

$$
\mathrm{d}(p, q) = \sqrt{ (p_{1} - q_{1})^{2} + \cdots + (p_{n}- q_{n})^{2} }
$$

Propiedades:
- Intuitiva: Distancia "en línea recta"
- Bien establecida: Funciona bien en muchos casos
- Eficiente: Cálculos optimizados disponibles
- Hipótesis: Todas las dimensiones son comparables

Para MNIST:
```python
# Dos imágenes de dígitos
imagen_1 = X_scaled[0]  # (784,)
imagen_2 = X_scaled[1]  # (784,)

# Distancia euclidiana
dist = np.sqrt(np.sum((imagen_1 - imagen_2)**2))
```

*Métricas alternativas disponibles:*

1. Manhattan (L1)
```python
metric='manhattan'
# d(p,q) = |p₁-q₁| + |p₂-q₂| + ... + |pₙ-qₙ|
```
- Uso: Datos con coordenadas independientes
- Ventaja: Más robusto a outliers que euclidiana

2. Cosine (Similitud de Coseno):
```python
metric='cosine'
# d(p,q) = 1 - (p·q)/(||p|| ||q||)
```
- Uso: Cuando la dirección importa más que la magnitud
- Aplicación: Text mining, análisis de documentos
- MNIST: Útil si nos interesa el "patrón" del dígito, no la intensidad

3. Correlation:
```python
metric='correlation'
# Similar a cosine pero centra los vectores primero
```
- Uso: Cuando las desviaciones de la media son importantes
- Aplicación: Series temporales, expresión genética

4. Hamming:
```python
metric='hamming'
# Porcentaje de características que difieren
```
- Uso: Datos binarios o categóricos
- Aplicación: Secuencias, one-hot encodings

#### `random_state` (Reproducibilidad)

*¿Por qué 42?*
- Común en ML: Valor ampliamente usado en ejemplos
- Arbitrario: Cualquier número funciona igual (0, 123, 2024, etc.)

*¿Cuándo importa random_state?:*
- Reproducibilidad: Papers, reportes, debugging
- Comparaciones: Evaluar diferentes parámetros consistentemente
- Colaboración: Otros pueden replicar resultados
- $\times$ Análisis exploratorio inicial: Menos crítico


#### **Resumen:**
```python
# 1. Configuración
reducer = umap.UMAP(
    n_neighbors=15,      # Control local-global
    min_dist=0.1,        # Control de compactación
    n_components=2,      # Dimensionalidad de salida
    metric='euclidean',  # Función de distancia
    random_state=42      # Reproducibilidad
)

# 2. Entrenamiento y transformación
embedding = reducer.fit_transform(X_scaled)

# 3. Resultado
Shape: (10000, 2)
Tipo: numpy.ndarray
Listo para visualización
```

Flujo interno de UMAP:

```
Entrada: X_scaled (10000, 784)

[1] Búsqueda de vecinos (KNN)
    
[2] Cálculo de distancias locales
    
[3] Construcción de grafo fuzzy
    
[4] Inicialización espectral
    
[5] Optimización SGD (n_epochs)
    
Salida: embedding (10000, 2)
```

**Parámetros clave resumidos:**

| Parámetro | Rango | Efecto Principal |
|-----------|-------|------------------|
| `n_neighbors` | 5-200 | Local $\iff$ Global |
| `min_dist` | 0.0-1.0 | Compacto $\iff$ Disperso |
| `n_components` | 2-50 | Visualización $\iff$ Features |
| `metric` | varios | Definición de similitud |

## Visualización del "Embedding" en R^2

In [ ]:
# Crear visualización interactiva del embedding UMAP con Plotly
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [ ]:
# Crear DataFrame para Plotly
df_embedding = pd.DataFrame({
    'UMAP_1': embedding[:, 0],
    'UMAP_2': embedding[:, 1],
    'Digito': Y_subset.astype(str),
    'Indice': range(len(Y_subset))
})

# Crear visualización interactiva
fig = px.scatter(
    df_embedding,
    x='UMAP_1',
    y='UMAP_2',
    color='Digito',
    title='Proyección de MNIST en R^2',
    hover_data=['Indice'],
    color_discrete_sequence=px.colors.qualitative.Set1,
    category_orders={'Digito': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']}
)

# Configurar diseño
fig.update_traces(marker_size=3, marker_opacity=1)
fig.update_layout(
    width=800,
    height=800,
    title_font_size=16,
    showlegend=True,
    legend_title_text='Dígito'
)

fig.show()

* **Visualización por Clase:**

In [ ]:
# Visualización separada por clase
fig, axes = plt.subplots(2, 5, figsize=(15, 8))
fig.suptitle('Embedding UMAP por Clase de Dígito', fontsize=16)

for digit in range(10):
    row = digit // 5
    col = digit % 5

    # Filtrar puntos de la clase actual
    mask = Y_subset == digit

    # Scatter plot de fondo (gris)
    axes[row, col].scatter(
        embedding[:, 0],
        embedding[:, 1],
        c='lightgray',
        s=1,
        alpha=0.3
    )

    # Scatter plot de la clase actual (coloreado)
    axes[row, col].scatter(
        embedding[mask, 0],
        embedding[mask, 1],
        c=f'C{digit}',
        s=3,
        alpha=0.8
    )

    axes[row, col].set_title(f'Dígito {digit}')
    axes[row, col].set_xticks([])
    axes[row, col].set_yticks([])

plt.tight_layout()
plt.show()

## Análisis de Parámetros UMAP

In [ ]:
!pip install colorama
from colorama import Fore

### `n_neighbors` (Vecindarios Locales)

*Conceptualmente:*
- Número de vecinos cercanos considerados para cada punto
- Escala local-global: Balance entre estructura local y global
- Construcción del grafo: Número de conexiones por nodo

*Funcionamiento interno:*

- Construcción del grafo de vecindarios:
```python
# Pseudocódigo del proceso
for punto_i in dataset:
    # Encontrar los k vecinos más cercanos
    vecinos = encontrar_k_vecinos_mas_cercanos(punto_i, k=15)
    
    # Crear aristas con pesos basados en distancias
    for vecino_j in vecinos:
        distancia = calcular_distancia(punto_i, vecino_j)
        peso = funcion_probabilidad(distancia)
        grafo.agregar_arista(punto_i, vecino_j, peso)
```

*Efectos de diferentes valores:*

n_neighbors pequeño (5-10):
- Preserva estructura local: Detalles finos visibles
- Clusters más granulares: Subclusters evidentes
- $\times$ Estructura global fragmentada: Conexiones a larga distancia perdidas
- $\times$ Más ruido: Sensible a variaciones locales

n_neighbors medio (15-30):
- Balance óptimo: Estructura local y global preservadas
- Robustez: Menos sensible a ruido
- Generalización: Captura patrones consistentes
- Recomendación: Valor por defecto para la mayoría de casos

n_neighbors grande (50-200):
- Estructura global clara: Relaciones macro preservadas
- Smooth embeddings: Transiciones suaves entre clusters
- $\times$ Pérdida de detalles locales: Estructuras finas comprimidas
- $\times$ Over-smoothing: Posible fusión de clusters distintos

Analogía:
Imagina que estás en una ciudad desconocida:
- n_neighbors=5: Solo preguntas a los 5 vecinos más cercanos $\to$ conoces tu barrio detalladamente
- n_neighbors=15: Preguntas a 15 personas $\to$ entiendes el barrio y algo de la ciudad
- n_neighbors=50: Preguntas a 50 personas $\to$ tienes mapa general pero pierdes detalles locales

Impacto en MNIST:
```python
# Ejemplo visual del efecto
n_neighbors_values = [5, 15, 50, 100]

# Con n_neighbors=5:  Dígitos similares forman sub-clusters muy separados
# Con n_neighbors=15: Balance óptimo, clusters claros pero conectados
# Con n_neighbors=50: Visión macro, transiciones suaves entre dígitos
```

In [ ]:
##### Comparar diferentes valores de n_neighbors #####
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
n_neighbors_values = [5, 15, 50]

for i, n_neighbors in enumerate(n_neighbors_values):
    print(Fore.RED + f"UMAP con n_neighbors={n_neighbors}...")

    reducer_temp = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=0.1,
        random_state=42
    )

    embedding_temp = reducer_temp.fit_transform(X_subset)

    scatter = axes[i].scatter(
        embedding_temp[:, 0],
        embedding_temp[:, 1],
        c=Y_subset,
        cmap='tab10',
        s=5,
        alpha=0.7
    )

    axes[i].set_title(f'n_neighbors = {n_neighbors}')
    axes[i].set_xlabel('UMAP 1')
    axes[i].set_ylabel('UMAP 2')

plt.tight_layout()
plt.show()

In [ ]:
##### Analizar la estructura de vecindarios #####
from sklearn.neighbors import NearestNeighbors

# Encontrar vecinos más cercanos en el espacio original
nn_original = NearestNeighbors(n_neighbors=15, metric='euclidean')
nn_original.fit(X_subset)

# Seleccionar un punto de ejemplo
example_idx = 1 # Puedes cambar de punto
distances, indices = nn_original.kneighbors([X_subset[example_idx]])

# Visualizar vecinos en espacio original
fig, axes = plt.subplots(1, 6, figsize=(15, 3))
fig.suptitle(f'Vecinos Más Cercanos del Punto {example_idx} (Dígito {Y_subset[example_idx]})')

for i in range(6):
    neighbor_idx = indices[0][i]
    axes[i].imshow(X_subset[neighbor_idx].reshape(28, 28), cmap='gray')
    axes[i].set_title(f'#{i}: {Y_subset[neighbor_idx]}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Mostrar posición en embedding UMAP
plt.figure(figsize=(10, 8))
plt.scatter(embedding[:, 0], embedding[:, 1], c='lightgray', s=5, alpha=0.5)
plt.scatter(embedding[indices[0], 0], embedding[indices[0], 1],
           c='orange', s=100, marker='o', label='Vecinos')
plt.scatter(embedding[example_idx, 0], embedding[example_idx, 1],
           c='red', s=100, marker='*', label='Punto Ejemplo')
plt.legend()
plt.title('Vecinos en el Embedding UMAP')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.grid(True, alpha=0.3)
plt.show()

### `min_dist` (Distancia Mínima)

*Conceptualmente:*
- Distancia mínima entre puntos en el embedding de baja dimensión
- Compactación vs. dispersión: Qué tan "apretados" están los clusters
- Solo afecta el embedding: No afecta la topología "aprendida"

*Funcionamiento interno:*

```python
# Pseudocódigo simplificado
def optimizar_embedding(grafo, min_dist):
    # Inicializar posiciones aleatorias en 2D
    posiciones = inicializar_aleatoriamente()
    
    for iteracion in range(n_epochs):
        for arista in grafo:
            punto_a, punto_b = arista
            
            # Calcular distancia actual en embedding
            dist_actual = distancia(posiciones[punto_a], posiciones[punto_b])
            
            # Si son vecinos en el grafo:
            if son_vecinos(punto_a, punto_b):
                # Fuerza atractiva (quiere acercarlos)
                if dist_actual > min_dist:
                    aplicar_fuerza_atractiva()
            else:
                # Fuerza repulsiva (quiere alejarlos)
                if dist_actual < dist_minima_no_vecinos:
                    aplicar_fuerza_repulsiva()
```

*Efectos de diferentes valores:*

min_dist pequeño (0.0 - 0.1):
- Clusters compactos**: Puntos muy juntos dentro de clusters
- Separación clara: Clusters bien diferenciados
- Visualización nítida: Estructuras más definidas
- $\times$ Posible solapamiento interno: Detalles internos comprimidos

min_dist medio (0.1 - 0.5):
- Balance visual: Ni muy compacto ni muy disperso
- Estructura interna visible: Detalles dentro de clusters preservados
- Recomendación: Valor estándar por defecto

min_dist grande (0.5 - 1.0):**
- Distribución uniforme: Puntos más espaciados
- Topología global clara: Relaciones macro visibles
- $\times$ Clusters difusos: Límites menos definidos
- $\times$ Uso ineficiente del espacio: Visualización más grande de lo necesario

Analogía:
Imagina organizar personas en una sala:
- min_dist=0.0: Personas del mismo grupo pegadas hombro con hombro
- min_dist=0.1: Espacio personal cómodo dentro de grupos
- min_dist=0.5: Cada persona tiene bastante espacio alrededor

Resumen:
```
┌───────────────────────────────────────────────────┐
│  n_neighbors: "¿Con quién debo agruparme?"        │
│  min_dist:    "¿Qué tan cerca debo estar?"        │
└───────────────────────────────────────────────────┘
```

Impacto en MNIST:
```python
min_dist=0.0:  #Clusters muy densos, como "islas" compactas
min_dist=0.1:  #Clusters definidos con estructura interna visible
min_dist=0.5:  #Clusters más difusos, transiciones suaves
```

In [ ]:
##### Comparar diferentes valores de min_dist #####
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
min_dist_values = [0.0, 0.1, 0.5]

for i, min_dist in enumerate(min_dist_values):
    print(Fore.RED + f"UMAP con min_dist={min_dist}...")

    reducer_temp = umap.UMAP(
        n_neighbors=15,
        min_dist=min_dist,
        random_state=42
    )

    embedding_temp = reducer_temp.fit_transform(X_subset)

    scatter = axes[i].scatter(
        embedding_temp[:, 0],
        embedding_temp[:, 1],
        c=Y_subset,
        cmap='tab10',
        s=5,
        alpha=0.7
    )

    axes[i].set_title(f'min_dist = {min_dist}')
    axes[i].set_xlabel('UMAP 1')
    axes[i].set_ylabel('UMAP 2')

plt.tight_layout()
plt.show()

### `n_components` (Dimensionalidad de Salida)

*Definición:*
- Número de dimensiones en el espacio de embedding
- Reducción: 784 dimensiones $\to$ 2 dimensiones

*Opciones comunes:*

n_components=2:
- Visualización directa: Gráficas 2D fáciles de interpretar
- Exploración rápida: Scatter plots inmediatos
- $\times$ Pérdida de información: Mayor compresión
- Usos comunes: Análisis exploratorio, presentaciones, papers

n_components=3:
- Más información preservada: Menos compresión
- Visualización 3D: Gráficas rotables
- $\times$ Más complejo visualizar: Requiere herramientas 3D
- Tip: Cuando 2D no es suficientemente informativo

n_components=10-50:
- Máxima preservación: Estructura compleja preservada
- Preprocessing para ML: Input para clasificadores
- $\times$ No visualizable directamente: Requiere análisis adicional
- Usos comunes: Feature engineering, pipeline de ML

*Comparación cuantitativa:*
```python
# Información preservada (aproximado para MNIST)
n_components=2:   #~65% de varianza explicada
n_components=3:   #~75% de varianza explicada
n_components=10:  #~90% de varianza explicada
n_components=50:  #~98% de varianza explicada
```

*Consideraciones prácticas:*
```python
# Para visualización
n_components = 2  # Siempre la primera opción

# Para preprocessing de ML
n_components = min(50, n_samples // 10)  # "Regla de oro"

# Para análisis detallado
n_components = 3  # Permite visualización 3D interactiva
```

* **n_components = 2**

In [ ]:
##### UMAP con n_components = 2 #####
reducer_2d = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    n_components=2,
    random_state=42,
    verbose=True
)

embedding_2d = reducer_2d.fit_transform(X_subset)

fig, ax = plt.subplots(figsize=(12, 10))
scatter = ax.scatter(
    embedding_2d[:, 0],
    embedding_2d[:, 1],
    c=Y_subset,
    cmap='tab10',
    s=10,
    alpha=0.7,
    edgecolors='none'
)
ax.set_xlabel('UMAP 1', fontsize=14, fontweight='bold')
ax.set_ylabel('UMAP 2', fontsize=14, fontweight='bold')
ax.set_title(f'UMAP Embedding con n_components = 2',
             fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, linestyle='--')

# Colorbar con etiquetas
cbar = plt.colorbar(scatter, ax=ax, label='Dígito')
cbar.set_ticks(range(10))
cbar.set_ticklabels(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

plt.tight_layout()
plt.show()

* **n_components = 3**

In [ ]:
##### UMAP con n_components = 3 #####
reducer_3d = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    n_components=3,
    random_state=42,
    verbose=True
)

embedding_3d = reducer_3d.fit_transform(X_subset)

In [ ]:
df_3d = pd.DataFrame({
    'UMAP_1': embedding_3d[:, 0],
    'UMAP_2': embedding_3d[:, 1],
    'UMAP_3': embedding_3d[:, 2],
    'Dígito': Y_subset.astype(str)  # Convertir a string
})

# Crear figura con orden explícito de categorías
fig = px.scatter_3d(
    df_3d,
    x='UMAP_1',
    y='UMAP_2',
    z='UMAP_3',
    color='Dígito',
    labels={
        'UMAP_1': 'UMAP 1',
        'UMAP_2': 'UMAP 2',
        'UMAP_3': 'UMAP 3',
        'Dígito': 'Dígito'
    },
    title=f'UMAP Embedding 3D Interactivo (n_components = 3)',
    color_discrete_sequence=px.colors.qualitative.T10,  # Similar a 'tab10'
    category_orders={'Dígito': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']},  # CLAVE
    opacity=0.7,
    height=700
)

# Ajustar estilo
fig.update_traces(
    marker=dict(
        size=3,
        line=dict(width=0)
    )
)

# Configurar layout
fig.update_layout(
    font=dict(size=12),
    title_font=dict(size=16, family='Arial Black'),
    scene=dict(
        xaxis_title='UMAP 1',
        yaxis_title='UMAP 2',
        zaxis_title='UMAP 3',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.3)  # Ángulo de vista inicial
        )
    ),
    showlegend=True,
    legend=dict(
        title="Dígito",
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    )
)

fig.show()

* **n_components "alto"**

In [ ]:
##### UMAP con n_components alto #####
reducer_high = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    n_components=min(50, n_samples // 10),
    random_state=42,
    verbose=False
)

embedding_high = reducer_high.fit_transform(X_subset)
n_comp = embedding_high.shape[1]

# Crear figura
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Subplot 1: Primeras 2 componentes
scatter1 = axes[0].scatter(
    embedding_high[:, 0],
    embedding_high[:, 1],
    c=Y_subset,
    cmap='tab10',
    s=10,
    alpha=0.7,
    edgecolors='none'
)

axes[0].set_xlabel('UMAP 1', fontsize=12, fontweight='bold')
axes[0].set_ylabel('UMAP 2', fontsize=12, fontweight='bold')
axes[0].set_title(f'Primeras 2 de {n_comp} dimensiones',
                  fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, linestyle='--')

axes[0].text(0.5, 0.98, f'Mostrando solo 2 de {n_comp} dimensiones',
             ha='center', va='top', transform=axes[0].transAxes,
             bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8, pad=0.5),
             fontsize=10, fontweight='bold')

# Subplot 2: Varianza explicada
component_variance = np.var(embedding_high, axis=0)
cumulative_variance = np.cumsum(component_variance) / np.sum(component_variance) * 100

axes[1].plot(range(1, len(component_variance) + 1), cumulative_variance,
             'b-', linewidth=2, marker='o', markersize=4)
axes[1].axhline(y=90, color='r', linestyle='--', linewidth=1, alpha=0.7, label='90% varianza')
axes[1].axhline(y=95, color='g', linestyle='--', linewidth=1, alpha=0.7, label='95% varianza')
axes[1].set_xlabel('Número de Componentes', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Varianza Acumulada (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Varianza Explicada (Aproximada)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, linestyle='--')
axes[1].legend(loc='lower right')
axes[1].set_xlim(0, n_comp + 1)
axes[1].set_ylim(0, 105)

# Colorbar
cbar = plt.colorbar(scatter1, ax=axes, label='Dígito', fraction=0.046, pad=0.04)
cbar.set_ticks(range(10))
cbar.set_ticklabels(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

plt.suptitle(f'UMAP Embedding con n_components = {n_comp}',
             fontsize=16, fontweight='bold', y=1.02)  # y=1.02 en lugar de 1.00
plt.show()

## Conclusiones

El **Silhouette Score** (Coeficiente de Silueta) es una métrica utilizada para evaluar la calidad de los clusters en algoritmos de agrupamiento o clustering.

*¿Qué mide?*

Mide qué tan bien está asignado cada punto de datos a su cluster, evaluando dos aspectos:

1. Cohesión: Qué tan cerca está un punto de otros puntos en su mismo cluster
2. Separación: Qué tan lejos está un punto de los puntos en otros clusters

*Interpretación del valor:*

El Silhouette Score varía entre **-1 y 1**:

- **Cercano a 1**: El punto está bien asignado a su cluster (muy cerca de su cluster y lejos de otros)
- **Cercano a 0**: El punto está en el límite entre dos clusters (podría pertenecer a cualquiera)
- Negativo: El punto probablemente está asignado al cluster incorrecto

*Fórmula:*

Para cada punto $i$:
$$
s(i) = \frac{b(i) - a(i)}{\max\{a(i), b(i)\}},
$$
donde
- $a(i)$: distancia promedio del punto $i$ a todos los otros puntos en su mismo cluster
- $b(i)$: distancia promedio del punto $i$ al cluster más cercano (del que no es miembro)

*Uso práctico:*

El Silhouette Score es especialmente útil para:
- Determinar el número óptimo de clusters
- Comparar diferentes algoritmos de clustering
- Identificar puntos mal clasificados

Es una de las métricas más populares porque no requiere conocer las etiquetas verdaderas (es una medida no supervisada).

**Resumen:**

 1.0 │ Perfecto: Clusters muy separados y compactos

 0.7 │ Excelente: Estructura clara
        
 0.5 │ Bueno: Clusters razonables
        
 0.3 │ Regular: Estructura débil
        
 0.0 │ Malo: Clusters solapados
        
-0.3 │ Muy malo: Puntos en cluster incorrecto
        
-1.0 │ Pésimo: Clasificación totalmente incorrecta

**Analogía:**

Imagina una fiesta con diferentes grupos de amigos:

Silhouette = 0.9: Cada grupo está junto, hablando entre ellos, y muy alejado de otros grupos. Es fácil identificar quién pertenece a qué grupo.

Silhouette = 0.5: Los grupos están juntos pero hay algunas personas en el medio. Puedes identificar los grupos pero hay mezclados.

Silhouette = 0.1: Todos están mezclados. Es difícil saber quién pertenece a qué grupo.

Silhouette = -0.5: Las personas están más cerca de otros grupos que del suyo propio. Claramente están en el grupo equivocado.

In [ ]:
##### Calcular métricas de calidad del embedding #####
from sklearn.metrics import silhouette_score

# Silhouette Score
sil_score = silhouette_score(embedding, Y_subset)
print(f"Silhouette Score: {sil_score:.3f}")

# Preservación de vecindarios locales
def local_neighborhood_preservation(X_original, X_embedded, k=10):
    """Calcula qué tan bien se preservan los vecindarios locales"""

    # Vecinos en espacio original
    nn_orig = NearestNeighbors(n_neighbors=k+1)
    nn_orig.fit(X_original)
    _, neighbors_orig = nn_orig.kneighbors(X_original)

    # Vecinos en espacio embedido
    nn_emb = NearestNeighbors(n_neighbors=k+1)
    nn_emb.fit(X_embedded)
    _, neighbors_emb = nn_emb.kneighbors(X_embedded)

    # Calcular preservación promedio
    preservation_scores = []
    for i in range(len(X_original)):
        orig_neighbors = set(neighbors_orig[i][1:])  # Excluir el punto mismo
        emb_neighbors = set(neighbors_emb[i][1:])

        intersection = len(orig_neighbors.intersection(emb_neighbors))
        preservation = intersection / k
        preservation_scores.append(preservation)

    return np.mean(preservation_scores)

# Calcular preservación de vecindarios
preservation = local_neighborhood_preservation(X_subset, embedding, k=10)
print(f"Preservación de Vecindarios Locales: {preservation:.3f}")

# Resumen de resultados
print("\n" + "="*50)
print("RESUMEN DE UMAP EN MNIST")
print("="*50)
print(f"Muestras analizadas: {n_samples}")
print(f"Dimensión original: {X_subset.shape[1]}")
print(f"Dimensión final: {embedding.shape[1]}")
print(f"Silhouette Score: {sil_score:.3f}")
print(f"Preservación Local: {preservation:.3f}")
print("="*50)